In [27]:
import requests
from bs4 import BeautifulSoup

In [28]:
# h2 = naglowki
# p = tekst (notatki?)
# ul = pole, ktore wyglada jak lista, a elementy są w kropce
# li = to są elementy na liscie
# dl = cale pole, ktore zawiera dymki NPC

url = "https://warcraft.wiki.gg/wiki/Light%27s_Arsenal"
odpowiedz = requests.get(
    url,
    headers={"User-Agent": "WoW_PolishTranslationProject -> (reachable on your Discord: Loe'Aner)"},
    timeout=30
)
html = odpowiedz.text

soup = BeautifulSoup(html, "html.parser")

In [ ]:
tresc = soup.select_one("#bodyContent")

pods_misji = tresc.find(class_="infobox darktable questbox")
pods_misji_slownik = {}
if pods_misji:
    print("==========PODSUMOWANIE MISJI==========")
    tytul_misji   = pods_misji.find(class_="above-header")
    tytul_misji_t = tytul_misji.get_text()

    pods_misji_slownik["Tytuł"] = tytul_misji_t

    for ps in pods_misji.find_all("th"):
        naglowki_t = ps.get_text().strip()
        zawartosc = ps.find_next_sibling()
        if not zawartosc or zawartosc.name != "td":
            continue

        match naglowki_t:
            case "Start":      naglowki_t = "Start_NPC"
            case "End":        naglowki_t = "Koniec_NPC"
            case "Level":      naglowki_t = "Wymagany_Poziom"
            case "Experience": naglowki_t = "Doświadczenie"
            case "Rewards":    naglowki_t = "Nagrody"
            case "Previous":   naglowki_t = "Poprzednia_Misja"
            case "Next":       naglowki_t = "Następna_Misja"
        pods_misji_slownik[naglowki_t] = zawartosc.get_text().strip()
        
    print(pods_misji_slownik)
print("\n")

cele_misji = tresc.find(id="Objectives")
cele_misji_slownik = {"Główny": {}, "Podrzędny": {}}
if cele_misji:
    print("==========CELE MISJI==========")
    naglowki     = cele_misji.find_parent("h2")

    glowny_cel = naglowki.find_next_sibling("p")
    if glowny_cel:
        cele_misji_slownik["Główny"][1] = glowny_cel.get_text().strip()
        # glowny cel jest jeden; reszta to lista pod spodem

    lista_celi   = glowny_cel.find_next_sibling() # ============================== powinien znalezc 'ul'
    if lista_celi:
        podcele = lista_celi.find_all("li")
        for i, pc in enumerate(podcele, start=1):
            podrzedny_cel_t = pc.get_text().strip()
            cele_misji_slownik["Podrzędny"][i] = podrzedny_cel_t
    print(cele_misji_slownik)
print("\n")

opis_misji = tresc.find(id="Description")
opis_misji_slownik = {}
if opis_misji:
    print("==========OPIS MISJI==========")
    opis = opis_misji.find_parent("h2")

    licznik = 0
    while True:
        opis = opis.find_next_sibling()
        if opis is None or opis.name == "h2":
            break
        elif opis.name == "p":
            licznik += 1
            opis_misji_slownik[licznik] = opis.get_text().strip()
    print(opis_misji_slownik)
print("\n")

postep = tresc.find(id="Progress")
postep_slownik = {}
if postep:
    print("==========POSTĘP MISJI==========")
    p = postep.find_parent("h2")
    
    licznik = 0
    while True:
        p = p.find_next_sibling()
        if p is None or p.name == "h2":
            break
        elif p.name == "p":
            licznik += 1
            postep_slownik[licznik] = p.get_text().strip()
    print(postep_slownik)
print("\n")

zakonczenie = tresc.find(id="Completion")
zakonczenie_slownik = {}
if zakonczenie:
    print("==========ZAKOŃCZENIE MISJI==========")
    z = zakonczenie.find_parent("h2")

    licznik = 0
    while True:
        z = z.find_next_sibling()
        if z is None or z.name == "h2":
            break
        elif z.name == "p":
            licznik += 1
            zakonczenie_slownik[licznik] = z.get_text().strip()
    print(zakonczenie_slownik)
print("\n")

nagroda = tresc.find(id="Rewards")
nagroda_slownik = {}
if nagroda:
    print("==========NAGRODA ZA MISJĘ==========")
    n = nagroda.find_parent("h2")

    licznik = 0
    while True:
        n = n.find_next_sibling()
        if n is None or n.name == "h2":
            break
        elif n.name == "p":
            licznik += 1
            nagroda_slownik[licznik] = n.get_text().strip()
    print(nagroda_slownik)

==========PODSUMOWANIE MISJI==========
{'Tytuł': "Light's Arsenal", 'Start_NPC': 'Lady Liadrin', 'Koniec_NPC': 'Lady Liadrin', 'Wymagany_Poziom': '80', 'Doświadczenie': '1,750', 'Nagrody': '23 40', 'Poprzednia_Misja': '[80]\xa0Severing the Void & \xa0[80]\xa0Voidborn Banishing', 'Następna_Misja': '[80]\xa0Wrath Unleashed'}


==========CELE MISJI==========
{'Główny': {1: 'Collect 7 Legendary Weapons.'}, 'Podrzędny': {1: 'Weapons collected (7)'}}


==========OPIS MISJI==========
{1: 'The armory here stores legendary weapons and artifacts, blessed with the power of the Light due to their proximity to the Sunwell.', 2: 'While we are pinned down by the Host. the ethereals are looting these treasures.', 3: '<Name>, secure these holy artifacts. They are our best weapons against the Void. Use them to cut down these invaders if you have the need.'}


==========POSTĘP MISJI==========
{1: 'Have you secured our artifacts?'}


==========ZAKOŃCZENIE MISJI==========
{1: 'Were they just here to loot u